In [1]:
# https://github.com/seungeunrho/RLfrombasics 참조

import random

In [2]:
# MC는 에피소드가 종료되어야 리턴을 알 수 있음 (실제 리턴값)
# TD는 에피소드가 종료되지 않아도 리턴을 알 수 있음 (추측 리턴값)
METHOD = 2 # 1 : MC(Monte Carlo), 2 : TD (Temporal Difference)
EPISODE = 50000

In [3]:
# environment class
class GridWorld():
    def __init__(self):
        self.x = 0
        self.y = 0
        
    def step(self, a):
        if a == 0:
            self.move_right()
        elif a == 1:
            self.move_left()
        elif a == 2:
            self.move_up()
        elif a == 3:
            self.move_down()
            
        reward = -1
        done = self.is_done()
        return (self.x, self.y), reward, done
    
    def move_right(self):
        self.y += 1
        if self.y > 3:
            self.y = 3
            
    def move_left(self):
        self.y -= 1
        if self.y < 0:
            self.y = 0
            
    def move_up(self):
        self.x -= 1
        if self.x < 0:
            self.x = 0
          
    def move_down(self):
        self.x += 1
        if self.x > 3:
            self.x = 3
            
    def is_done(self):
        if self.x == 3 and self.y == 3:
            return True
        else:
            return False
        
    def get_state(self):
        return (self.x, self.y)
    
    def reset(self):
        self.x = 0
        self.y = 0
        return (self.x, self.y)
    


In [4]:
# Agent class
class Agent():
    def __init__(self):
        pass
    
    def select_action(self):
        coin = random.random()
        if coin < 0.25:
            action = 0
        elif coin < 0.5:
            action = 1
        elif coin < 0.75:
            action = 2
        else:
            action = 3
            
        return action
    


In [5]:
def main():
    env = GridWorld()
    agent = Agent()
    data = [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]] # 테이블 초기화
    gamma = 1.0
    
    if METHOD == 1:
        alpha = 0.0001
        for k in range(EPISODE): # 총 5만번의 에피소드 진행
            done = False
            history = []
            while not done:
                action = agent.select_action()
                (x,y), reward, done = env.step(action)
                history.append((x,y,reward))
            env.reset()    
            
            #print(history)
            
            # 매 에피소드가 끝나고 바로 해당 데이터를 이용해 테이블을 얻데이트
            cum_reward = 0
            for transition in history[::-1]:
                # 방문했던 상태들을 뒤에서부터 차례차례 리턴을 계산
                x, y, reward = transition
                #print(transition)

                # 최소 수식 V(St) = (1 - alpha) * V(St) + alpha * Gt
                # 변경 수식 V(St) = V(St) + alpha * (Gt - V(St))
                # V(St)에 대한 값을 근사치에 가까이 가게하기 위하여 EPISODE 많이 해주어야 함
                # alpha(0~1 사이 값)의 값이 클수록 경험값이 한번에 크게 업데이트 되고
                # alpha의 값이 작을수록 경험값이 조금씩(보수적으로) 업데이트 됨
                data[x][y] = data[x][y] + alpha*(cum_reward - data[x][y])

                # 수식 Gt = Rt+1 + r * Gt+1
                # G0 = R0 + gamma*R1 + .... + gamma의 t-1 제곱 * Rt-1
                # Gt-1 = R1
                # Gt = 0 -> Gt는 종료 State
                # cum_reward는 Gt
                # gamma는 감쇠인자. 먼 미래일 경우 보상 값을 적게하기 위함
                cum_reward = cum_reward + gamma * reward
    else:
        alpha = 0.01 # MC보다 큰 값을 사용
        for k in range(EPISODE): # 총 5만번의 에피소드 진행
            done = False
            while not done:
                x, y = env.get_state()
                action = agent.select_action()
                (x_prime, y_prime), reward, done = env.step(action)
                #print(x_prime, y_prime)

                # V(St)에 대한 값을 근사치에 가까이 가게하기 위하여 EPISODE 많이 해주어야 함
                # alpha(0~1 사이 값)의 값이 클수록 경험값이 한번에 크게 업데이트 되고
                # alpha의 값이 작을수록 경험값이 조금씩(보수적으로) 업데이트 됨                
                # gamma는 감쇠인자. 먼 미래일 경우 보상 값을 적게하기 위함
                # 한 번의 step이 진행되자마자 바로 테이블의 데이터를 업데이트 해줌
                # 수식 V(St) = V(St) + alpha * (Rt+1 + gamma * V(St+1) - V(St))
                data[x][y] = data[x][y] + alpha*(reward + gamma * data[x_prime][y_prime] - data[x][y])

            env.reset()    

    # 학습이 끝나고 난 후 데이터를 출력해 보기 위한 코드
    for row in data:
        print(row)
        
    
    

In [6]:
main()

[-59.14116007273648, -57.35847726920523, -54.33440637231192, -51.0320059405729]
[-57.14057072359945, -54.00313261218074, -48.846343053536444, -44.38458593403774]
[-54.02643211956618, -49.61695994391938, -40.68415444793987, -28.839126767513836]
[-51.93090658473838, -45.368134627808764, -27.756643685960405, 0]
